## Import Libraries

In [222]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime as dt
import numpy as np
import math

## Load dataset

In [223]:
df = pd.read_csv('SampleAssessment.csv')
print(df.shape)
print(df.columns.tolist())
print(df.dtypes)

(10000, 11)
['customer_id', 'First Time', 'Recent Time', '# of Orders', '# of Orders in last 7 days', '# of Orders in last 4 weeks', 'Amount', 'Amount in last 7 days', 'Amount in last 4 weeks', 'Avg_DistanceFromResturant', 'Avg_DeliveryTime']
customer_id                      int64
First Time                      object
Recent Time                     object
# of Orders                      int64
# of Orders in last 7 days     float64
# of Orders in last 4 weeks    float64
Amount                           int64
Amount in last 7 days            int64
Amount in last 4 weeks           int64
Avg_DistanceFromResturant      float64
Avg_DeliveryTime                 int64
dtype: object


## Convert object to time

In [224]:
cols = ['First Time', 'Recent Time']
for column in cols:
    df[column] = pd.to_datetime(df[column])
df.dtypes

customer_id                             int64
First Time                     datetime64[ns]
Recent Time                    datetime64[ns]
# of Orders                             int64
# of Orders in last 7 days            float64
# of Orders in last 4 weeks           float64
Amount                                  int64
Amount in last 7 days                   int64
Amount in last 4 weeks                  int64
Avg_DistanceFromResturant             float64
Avg_DeliveryTime                        int64
dtype: object

## count of unique vs null values

In [225]:
# summary_df = pd.DataFrame(columns=['num_unique', 'num_nulls'])
# for column in df.columns:
#     summary_df.loc[column] = [df[column].nunique(), df[column].isna().sum()]
# summary_df

### searching for null values

In [226]:
df.isnull().sum(axis=0)

customer_id                       0
First Time                        0
Recent Time                       0
# of Orders                       0
# of Orders in last 7 days     8077
# of Orders in last 4 weeks    5659
Amount                            0
Amount in last 7 days             0
Amount in last 4 weeks            0
Avg_DistanceFromResturant         0
Avg_DeliveryTime                  0
dtype: int64

### Only the below columns have null values, replace nulls with zeros
* '# of Orders in last 7 day'
* '# of Orders in last 4 weeks'

In [227]:
cols = ['# of Orders in last 7 days', '# of Orders in last 4 weeks']
for column in cols:
    df[column].fillna(0, inplace=True)
print(df[df['# of Orders in last 7 days'].isna()].shape)
print(df[df['# of Orders in last 4 weeks'].isna()].shape)

(0, 11)
(0, 11)


## Adding average order value column

In [228]:
df['average_cost'] = df.Amount / df['# of Orders']

## Adding a time difference column

In [229]:
df['time_diff'] =  df['Recent Time'] - df['First Time']

In [230]:
df.sample(5)

,customer_id,First Time,Recent Time,# of Orders,# of Orders in last 7 days,# of Orders in last 4 weeks,Amount,Amount in last 7 days,Amount in last 4 weeks,Avg_DistanceFromResturant,Avg_DeliveryTime,average_cost,time_diff
3574,254949,2015-07-21 04:37:00,2015-08-29 20:10:00,4,0.0,0.0,1079,0,0,1.8,24,269.750000,39 days 15:33:00
4640,627893,2015-09-21 16:33:00,2015-12-22 07:33:00,3,0.0,2.0,1286,0,1020,2.8,61,428.666667,91 days 15:00:00
6938,1047998,2015-11-18 08:36:00,2015-11-22 08:27:00,1,0.0,1.0,409,0,409,2.3,26,409.000000,3 days 23:51:00
3498,395690,2015-08-04 10:45:00,2015-11-09 23:11:00,4,0.0,0.0,1439,0,0,2.4,43,359.750000,97 days 12:26:00
5864,326302,2015-06-28 08:10:00,2015-10-07 11:33:00,2,0.0,0.0,1483,0,0,2.3,46,741.500000,101 days 03:23:00


In [231]:
df.columns

Index(['customer_id', 'First Time', 'Recent Time', '# of Orders',
       '# of Orders in last 7 days', '# of Orders in last 4 weeks', 'Amount',
       'Amount in last 7 days', 'Amount in last 4 weeks',
       'Avg_DistanceFromResturant', 'Avg_DeliveryTime', 'average_cost',
       'time_diff'],
      dtype='object')

In [232]:
print(df['First Time'].min())
print(df['First Time'].max())

print(df['Recent Time'].min())
print(df['Recent Time'].max())

2015-06-05 17:21:00
2016-01-07 12:42:00
2015-06-08 22:07:00
2016-01-13 05:54:00


## Generating RMF metrics

In [233]:
rename_dict = {"# of Orders": "Frequency", "Amount": "Monetory"}
df.rename(rename_dict, inplace=True, axis=1)
x = (dt.datetime.utcnow() - df['Recent Time'])
df['Recency'] = df['Recent Time'].apply(lambda x: (dt.datetime.utcnow() - x).days)

In [234]:
df.dtypes

customer_id                              int64
First Time                      datetime64[ns]
Recent Time                     datetime64[ns]
Frequency                                int64
# of Orders in last 7 days             float64
# of Orders in last 4 weeks            float64
Monetory                                 int64
Amount in last 7 days                    int64
Amount in last 4 weeks                   int64
Avg_DistanceFromResturant              float64
Avg_DeliveryTime                         int64
average_cost                           float64
time_diff                      timedelta64[ns]
Recency                                  int64
dtype: object

## Creating quantiles

In [235]:
quantile_baskets = [0.2, 0.4, 0.6, 0.8, 1]
df_quantiles_dict = df.quantile(quantile_baskets).to_dict()

In [236]:
print(df_quantiles_dict)

{'customer_id': {0.2: 267988.4, 0.4: 532701.8, 0.6: 802909.6, 0.8: 1075403.8, 1.0: 1355445.0}, 'Frequency': {0.2: 1.0, 0.4: 2.0, 0.6: 3.0, 0.8: 9.0, 1.0: 212.0}, '# of Orders in last 7 days': {0.2: 0.0, 0.4: 0.0, 0.6: 0.0, 0.8: 0.0, 1.0: 14.0}, '# of Orders in last 4 weeks': {0.2: 0.0, 0.4: 0.0, 0.6: 1.0, 0.8: 2.0, 1.0: 46.0}, 'Monetory': {0.2: 218.0, 0.4: 495.0, 0.6: 1036.0, 0.8: 2689.2000000000007, 1.0: 138808.0}, 'Amount in last 7 days': {0.2: 0.0, 0.4: 0.0, 0.6: 0.0, 0.8: 0.0, 1.0: 10150.0}, 'Amount in last 4 weeks': {0.2: 0.0, 0.4: 0.0, 0.6: 66.0, 0.8: 567.0, 1.0: 26853.0}, 'Avg_DistanceFromResturant': {0.2: 1.5, 0.4: 2.1, 0.6: 2.6, 0.8: 3.2, 1.0: 5.9}, 'Avg_DeliveryTime': {0.2: 24.0, 0.4: 32.0, 0.6: 41.0, 0.8: 50.0, 1.0: 83.0}, 'average_cost': {0.2: 111.0, 0.4: 221.65882352941185, 0.6: 348.4571428571428, 0.8: 535.3632183908047, 1.0: 9906.5}, 'Recency': {0.2: 1013.0, 0.4: 1031.0, 0.6: 1060.0, 0.8: 1102.0, 1.0: 1196.0}}


In [237]:
def r_score(curr_value, quantile_dict):
    for index, quantile in quantile_dict.items():
        if curr_value <= quantile:
            return quantile_baskets.index(index) + 1
    
def f_m_score(curr_value, quantile_dict):
    for index, quantile in quantile_dict.items():
        if curr_value <= quantile:
            return len(quantile_dict) - quantile_baskets.index(index)

In [238]:
df['R'] = df['Recency'].apply(lambda x: r_score(x, df_quantiles_dict['Recency']))
df['M'] = df['Monetory'].apply(lambda x: f_m_score(x, df_quantiles_dict['Monetory']))
df['F'] = df['Frequency'].apply(lambda x: f_m_score(x, df_quantiles_dict['Frequency']))
df['RMF'] = df.R.apply(str) + df.M.apply(str) + df.F.apply(str)

In [239]:
df_quantiles_dict['Recency'].items()

dict_items([(0.2, 1013.0), (0.4, 1031.0), (0.6, 1060.0), (0.8, 1102.0), (1.0, 1196.0)])

In [240]:
df.sample(5)

,customer_id,First Time,Recent Time,Frequency,# of Orders in last 7 days,# of Orders in last 4 weeks,Monetory,Amount in last 7 days,Amount in last 4 weeks,Avg_DistanceFromResturant,Avg_DeliveryTime,average_cost,time_diff,Recency,R,M,F,RMF
4529,462939,2015-07-25 17:00:00,2015-08-01 08:12:00,3,0.0,0.0,1749,0,0,2.5,55,583.000000,6 days 15:12:00,1142,5,2,3,523
9773,23889,2015-11-13 01:06:00,2015-11-14 22:40:00,1,0.0,1.0,32,0,32,1.5,36,32.000000,1 days 21:34:00,1037,3,5,5,355
9876,401275,2015-08-18 07:13:00,2015-08-21 11:29:00,1,0.0,0.0,413,0,0,2.4,15,413.000000,3 days 04:16:00,1122,5,4,5,545
4704,475911,2015-08-31 22:23:00,2015-12-31 05:19:00,3,0.0,2.0,4648,0,663,2.3,51,1549.333333,121 days 06:56:00,991,1,1,3,113
2383,922092,2015-11-01 09:35:00,2015-11-05 20:51:00,7,0.0,0.0,357,0,0,2.1,51,51.000000,4 days 11:16:00,1046,3,4,2,342


In [241]:
df[df['RMF'] == '111'].shape

(632, 18)

### We have decent number of customers to target in the first basket
### Explore the baskets having RMF 555, looks like they have placed only one 

In [247]:
df[df['RMF'] == '555']

,customer_id,First Time,Recent Time,Frequency,# of Orders in last 7 days,# of Orders in last 4 weeks,Monetory,Amount in last 7 days,Amount in last 4 weeks,Avg_DistanceFromResturant,Avg_DeliveryTime,average_cost,time_diff,Recency,R,M,F,RMF,mean,7_days_propensity
6365,1162834,2015-08-02 09:15:00,2015-08-02 21:05:00,1,0.0,0.0,99,0,0,0.9,34,99.0,0 days 11:50:00,1141,5,5,5,555,0.0,0.000000
6392,574614,2015-08-23 19:30:00,2015-08-26 06:48:00,1,0.0,0.0,167,0,0,0.4,29,167.0,2 days 11:18:00,1118,5,5,5,555,2.0,0.003437
6399,1214823,2015-06-23 18:03:00,2015-06-26 14:28:00,1,0.0,0.0,109,0,0,2.0,35,109.0,2 days 20:25:00,1178,5,5,5,555,2.0,0.003437
6408,679331,2015-08-16 00:16:00,2015-08-20 22:27:00,1,0.0,0.0,86,0,0,3.7,44,86.0,4 days 22:11:00,1123,5,5,5,555,4.0,0.059540
6413,575306,2015-09-09 09:34:00,2015-09-09 10:32:00,1,0.0,0.0,167,0,0,3.7,32,167.0,0 days 00:58:00,1103,5,5,5,555,0.0,0.000000
6415,168077,2015-06-18 07:38:00,2015-06-18 11:58:00,1,0.0,0.0,158,0,0,1.1,40,158.0,0 days 04:20:00,1186,5,5,5,555,0.0,0.000000
6426,456971,2015-07-14 22:27:00,2015-07-19 14:28:00,1,0.0,0.0,60,0,0,0.6,27,60.0,4 days 16:01:00,1155,5,5,5,555,4.0,0.059540
6459,642126,2015-08-18 09:54:00,2015-08-22 06:11:00,1,0.0,0.0,158,0,0,1.3,27,158.0,3 days 20:17:00,1122,5,5,5,555,3.0,0.021604
6467,428629,2015-07-31 11:14:00,2015-07-31 19:02:00,1,0.0,0.0,45,0,0,2.6,32,45.0,0 days 07:48:00,1143,5,5,5,555,0.0,0.000000
6469,418352,2015-06-25 09:51:00,2015-06-27 19:51:00,1,0.0,0.0,194,0,0,0.9,50,194.0,2 days 10:00:00,1177,5,5,5,555,2.0,0.003437


In [243]:
window = 7

den = math.factorial(window)
def buyer_propensity(mean, window):
    return np.exp(-mean) * np.power(mean, window) / den


df['mean'] = df.time_diff.apply(lambda x: x.days) / df.Frequency
df['7_days_propensity'] = df['mean'].apply(lambda x : buyer_propensity(x, window))

In [244]:
df[df['7_days_propensity'] > 0.1].shape

(1637, 20)